In [1]:
import initialize_notebook
import pandas as pd
from ams.DateRange import DateRange
%matplotlib inline
from pathlib import Path

from pyspark.sql import functions as F

from ams.config import constants
from ams.services import spark_service
pd.set_option("display.max_rows", 101)

In [2]:
spark = spark_service.get_or_create('assign_sentiment_and_stock')

In [3]:
twitter_folder = 'twitter'

learning_prep_path = Path(constants.DATA_PATH, twitter_folder, "learning_prep_drop", "learning_prep_drop_2020-08-31_03-22-33-670.81", "part-00000-d3be8a07-df94-4097-a5d1-cb2d7836f130-c000.csv")

In [4]:
df_twitter = pd.read_csv(str(learning_prep_path))
# df_red = df.limit(10000)
# df_red = df_pd.drop(*['created_at', 'created_at_timestamp'])
# df_red = df.sample(fraction=.01)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,16,23,26,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
from ams.services import ticker_service

tickers = df_twitter['f22_ticker'].unique().tolist()
# tickers = [single_ticker]

Setting up logging...
Will use logging path: /home/jovyan/work/data/logs/alpha_media_signal


In [6]:
from pyspark.sql import functions as F

date_range = DateRange.from_date_strings(from_date_str="2020-08-01", to_date_str="2020-08-27")
df_stock_data = ticker_service.get_tickers_in_range(tickers=tickers, date_range=date_range)
print(f'{df_stock_data.shape[0]}')

41173


In [8]:
df_stock_renamed = df_stock_data.rename(columns={"ticker": "f22_ticker"})

df_stock_clean = df_stock_renamed.drop(columns=['None', 'lastupdated'])
print(f'Twitter: {df_twitter.shape[0]}')
print(f'Quandl: {df_stock_renamed.shape[0]}')

print(f'{df_twitter.columns}')
print(f'{df_stock_renamed.columns}')

df_merged = pd.merge(df_twitter, df_stock_renamed, how='inner', left_on=["f22_ticker","date"], right_on=["f22_ticker","date"])
print(f'Num merged: {df_merged.shape[0]}')

Twitter: 907029
Quandl: 41173
Index(['created_at', 'created_at_timestamp', 'user_screen_name',
       'favorite_count', 'in_reply_to_screen_name', 'user_verified',
       'metadata_result_type', 'user_listed_count', 'user_time_zone',
       'user_has_extended_profile', 'in_reply_to_status_id',
       'user_statuses_count', 'user_protected', 'user_is_translation_enabled',
       'user_location', 'lang', 'user_geo_enabled', 'place_country',
       'place_name', 'possibly_sensitive', 'user_friends_count',
       'retweet_count', 'user_follow_request_sent', 'user_followers_count',
       'f22_ticker', 'f22_has_cashtag', 'f22_ticker_in_text',
       'f22_num_other_tickers_in_tweet', 'f22_sentiment_pos',
       'f22_sentiment_neu', 'f22_sentiment_neg', 'f22_sentiment_compound',
       'f22_compound_score', 'date'],
      dtype='object')
Index(['f22_ticker', 'date', 'open', 'high', 'low', 'close', 'volume',
       'dividends', 'closeunadj', 'lastupdated', 'None'],
      dtype='object')
Num me

In [ ]:
# Necessary for K-means to for first preds.
df_merged_mean = (df_merged[(df_merged["f22_ticker"].isin(tickers))].groupby(by=[df_merged["date"], df_merged["f22_ticker"]]).mean())
# print(df_twitter_mean.columns)
print(df_merged_mean.shape[0])

df_blue_rabbit = df_merged_mean

In [ ]:
from sklearn.preprocessing import StandardScaler

df_blue_rabbit['stock_val_change'] = (df_blue_rabbit['close'] - df_blue_rabbit['open']) / df_blue_rabbit['open'] * 100.0

scaler = StandardScaler()
df_blue_rabbit['stock_val_change_scaled'] = scaler.fit_transform(df_blue_rabbit[['stock_val_change']])
print(df_blue_rabbit.columns)


df_blue_rabbit.head(1000)

# forecast_out = int(math.ceil(0.013 * df_blue_rabbit.shape[0]))

tick_tups = df_blue_rabbit.reset_index(['f22_ticker', 'date'])[['f22_ticker', 'date']].values.tolist()

print(tick_tup[:3])
for tt in tick_tups:
    # Look up high price in forecast_out days.
    pass

In [ ]:
import math

forecast_col = 'stock_val_change'
forecast_out = 3
print(f'Forecast out: {forecast_out}')
df_blue_rabbit['stock_val_change_pred'] = df_blue_rabbit[forecast_col].shift(-forecast_out)
df_blue_rabbit.head(2)

In [ ]:
df_blue_rabbit['buy_sell'] = df_blue_rabbit['stock_val_change_pred'].apply(lambda x: 1 if x >=0 else -1)

for c in df_blue_rabbit.columns:
    df_blue_rabbit[c] = df_blue_rabbit[c].fillna(0)

df_blue_rabbit.head(2)



In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

train_set, test_set = train_test_split(df_blue_rabbit, test_size=0.2, random_state=42, shuffle=True)

train_cols = ['created_at_timestamp', 'favorite_count', 'user_verified',
       'user_listed_count', 'user_time_zone', 'in_reply_to_status_id',
       'user_statuses_count', 'user_protected', 'user_is_translation_enabled',
       'user_friends_count', 'retweet_count', 'user_follow_request_sent',
       'f22_has_cashtag', 'f22_num_other_tickers_in_tweet',
       'f22_sentiment_pos', 'f22_sentiment_neu', 'f22_sentiment_neg',
       'f22_sentiment_compound']

X_train = np.array(train_set[train_cols])
X_test = np.array(test_set[train_cols])

y_train = np.array(train_set['buy_sell'])
y_test = np.array(test_set['buy_sell'])

In [ ]:
scaler = StandardScaler()
X_train_std= scaler.fit_transform(X_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

for i in range(1,1500):
    knn = KNeighborsClassifier(n_neighbors=i).fit(X_train_std, y_train)
    y_test_pred = knn.predict(X_test)
    if i % 20 == 0:
        print(accuracy_score(y_test, y_test_pred))

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

for i in range(1,5):
    svc= SVC(kernel="linear", random_state=0, gamma=i)
    model= svc.fit(X_train_std, y_train)
    scores = cross_val_score(estimator=model, X=X_train_std, y=y_train, cv=5)
    print(i, ':', np.average(scores))

In [ ]:
def train_svc(kernel_name: str):
    for i in range(1,5):
        svc= SVC(kernel=kernel_name, random_state=0, gamma=i)
        model= svc.fit(X_train_std, y_train)
        scores = cross_val_score(estimator=model, X=X_train, y=y_train, cv=5)
        print(i, ':', np.average(scores))

In [ ]:
train_svc(kernel_name='rbf')
train_svc(kernel_name='sigmoid')

In [ ]:
# Cross-validation
from sklearn.tree import DecisionTreeClassifier

decisiontree= DecisionTreeClassifier(max_depth=4)
model = decisiontree.fit(X_train_std, y_train)
y_test_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_test_pred)

#Add to the list
dtlist =[]
dtlist.append(y_test_pred)